<a href="https://colab.research.google.com/github/deepak121993/END/blob/main/Bert_Transformer_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/9479fcb532214ad26fd4bda9fcf081a05e1aaf4e/assets/transformer1.png)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [44]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 11.5MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=84fb733e7f75a2f72bc5d8958981ccc1040089c10298cfe2152b77c65e3bbd89
  Stored in directory: /tmp/pip-ephem-wheel-cache-80v49stk/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [45]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [5]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)]

# def tokenize_en(text):
#     """
#     Tokenizes English text from a string into a list of strings
#     """
#     return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
import pandas as pd
import numpy as np

df  =  pd.read_excel(r"/content/scrapped_600.xlsx")

In [7]:
# SRC = Field(tokenize = tokenize_en, 
#             init_token = '<sos>', 
#             eos_token = '<eos>', 
#             lower = True, 
#             batch_first = True)

# TRG = Field(tokenize = tokenize_en, 
#             init_token = '<sos>', 
#             eos_token = '<eos>', 
#             lower = True, 
#             batch_first = True)


In [8]:
df.head()

,Term,Description,KeyTakeAways,link
0,What Is an A-B Trust?,An A-B trust is a joint trust created by a mar...,An A-B trust minimizes estate taxes by splitti...,https://www.investopedia.com/terms/a/a-b-trust...
1,What Are China A-Shares?,China A-shares are the stock shares of mainlan...,China A-shares are the stock shares of mainlan...,https://www.investopedia.com/terms/a/a-shares.asp
2,What Is AAA?,AAA is the highest possible rating that may be...,The highest possible rating that a bond may ac...,https://www.investopedia.com/terms/a/aaa.asp
3,What Is Average Annual Growth Rate (AAGR)?,The average annual growth rate (AAGR) is the a...,This ratio helps you figure out how much avera...,https://www.investopedia.com/terms/a/aagr.asp
4,What Is the Average Annual Return (AAR)?,The average annual return (AAR) is a percentag...,The average annual return (AAR) is a percentag...,https://www.investopedia.com/terms/a/aar.asp


In [9]:
# train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
#                                                     fields = (SRC, TRG))
# from torchtext import data

# t_src = df['Term'].apply(lambda x : tokenize_en(x))
# t_trg = df['Description'].apply(lambda x : tokenize_en(x))

# TRG = data.Field(lower=True, tokenize=t_trg.tokenizer, init_token='<sos>', eos_token='<eos>')
# SRC = data.Field(lower=True, tokenize=t_src.tokenizer)

def remove_punc(string):
    punctuations = '''!()[]{};:'"\,<>./?@#$%^&*~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

src = list(df['Term'])
trg = list(df['Description'])


pairs = []
max_len = 25

for i in range(len(src)-1):

  qa_pairs = []
  p = remove_punc(src[i])
  q = remove_punc(trg[i])

  qa_pairs.append(p.split()[:max_len])
  qa_pairs.append(q.split()[:max_len])
  pairs.append(qa_pairs)

In [10]:
# SRC.build_vocab(train_data, min_freq = 2)
# TRG.build_vocab(train_data, min_freq = 2)


######## building vocab ##########
word_freq = Counter()

for pair in pairs:
  word_freq.update(pair[0])
  word_freq.update(pair[1])

min_word_freq = 5

words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq ]

word_map = { k : v + 1 for v, k in enumerate(words) }
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

In [11]:
print("Total words are: {}".format(len(word_map)))
with open('WORDMAP_corpus.json', 'w') as j:
    json.dump(word_map, j)

Total words are: 448


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
# BATCH_SIZE = 128

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
#     (train_data, valid_data, test_data), 
#      batch_size = BATCH_SIZE,
#      device = device)

def encode_question(words, word_map):

  enc_c = [word_map.get(word ,  word_map['<unk>']) for word in words] + [word_map['<pad>']]*(max_len - len(words))

  return enc_c

def encode_answer(words, word_map):

  enc_a = [word_map['<start>']] + [word_map.get(word ,  word_map['<unk>']) for word in words] +\
   [word_map['<end>']] + [word_map['<pad>']]*(max_len - len(words))

  return enc_a




In [14]:
pairs_encoded = []
for pair in pairs:

  qus = encode_question(pair[0] , word_map)
  ans = encode_answer(pair[1] , word_map)

  pairs_encoded.append([qus , ans])

In [15]:
with open('pairs_encoded.json', 'w') as p:
    json.dump(pairs_encoded, p)

class Dataset(Dataset):

  def __init__(self):

    self.pairs = json.load(open('pairs_encoded.json'))
    self.dataset_size = len(self.pairs)

  def __getitem__(self, i):

    question = torch.LongTensor(self.pairs[i][0])
    answer =   torch.LongTensor(self.pairs[i][1])

    return question , answer

  def __len__(self):
    
    return self.dataset_size


In [16]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size = 100,
                                           shuffle=True,
                                           pin_memory=True
                                           )


In [17]:
x , y = next(iter(train_loader))

In [18]:
x.shape , y.shape

(torch.Size([100, 25]), torch.Size([100, 27]))

In [19]:
class Embedding(nn.Module):

  ####  token dim  #####
  def __init__(self, vocab_size , d_model , max_len = 50):
    super(Embedding, self).__init__()
    self.d_model = d_model
    self.embed = nn.Embedding(vocab_size , d_model)
    self.dropout = nn.Dropout(0.1)
    self.pe  =  self.positional_embedding(max_len ,  d_model)

  def positional_embedding(self , max_len ,  d_model):
    pe  = torch.zeros(max_len , d_model).to(device)
    for pos in range(max_len):
      for i in range(0 , d_model ,2):
        pe[pos , i] = math.sin(pos / (10000 ** ((2 * i)/d_model))) 
        pe[pos ,  i+1] =  math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
    pe = pe.unsqueeze(0)  ## include the batch size 
    return pe

  def forward(self , encoded_words):
    embedding = self.embed(encoded_words) * math.sqrt(self.d_model)
    #print("embedding shape " , embedding.size())
    embedding += self.pe[: , embedding.size(1)]   # pe will automatically be expanded with the same batch size as encoded_words
    embedding = self.dropout(embedding)
    return embedding

In [20]:
class Encoder(nn.Module):

    def __init__(self,input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_length = 100):
      
        super().__init__()
        self.device = device
        self.embed = Embedding(input_dim, hid_dim)
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        batch_size = src.shape[0]
        src_len = src.shape[1]
        src = self.embed(src)
        for layer in self.layers:
            src = layer(src, src_mask)
        return src

In [21]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len]       
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        #src = [batch size, src len, hid dim]

        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [22]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
        
        Q = self.fc_q(query)  #Q = [batch size, query len, hid dim]
        K = self.fc_k(key)    #K = [batch size, key len, hid dim]
        V = self.fc_v(value)  #V = [batch size, value len, hid dim]
        
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)   #Q = [batch size, n heads, query len, head dim]
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)   #K = [batch size, n heads, key len, head dim]
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)   #V = [batch size, n heads, value len, head dim]
    
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale   #energy = [batch size, n heads, query len, key len]
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim = -1)   #attention = [batch size, n heads, query len, key len]
        x = torch.matmul(self.dropout(attention), V)  #x = [batch size, n heads, query len, head dim]
        x = x.permute(0, 2, 1, 3).contiguous()        #x = [batch size, query len, n heads, head dim]
        x = x.view(batch_size, -1, self.hid_dim)      #x = [batch size, query len, hid dim]
        x = self.fc_o(x)                              #x = [batch size, query len, hid dim]
        return x, attention

In [23]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [batch size, seq len, hid dim]
        x = self.dropout(torch.relu(self.fc_1(x)))
        #x = [batch size, seq len, pf dim]
        x = self.fc_2(x)
        #x = [batch size, seq len, hid dim]
        return x

![](https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/9479fcb532214ad26fd4bda9fcf081a05e1aaf4e/assets/transformer-decoder.png)

In [24]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device,max_length = 100):
        
        super().__init__()
        self.device = device
        self.embed = Embedding(output_dim, hid_dim)
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        trg = self.embed(trg)   #trg = [batch size, trg len, hid dim]
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)   #trg = [batch size, trg len, hid dim]  
                                                                       #attention = [batch size, n heads, trg len, src len] 
        output = self.fc_out(trg)    #output = [batch size, trg len, output dim]
        
        return output, attention

In [25]:
class DecoderLayer(nn.Module):

    def __init__(self, hid_dim, n_heads,pf_dim, dropout, device):
      
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)

        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]

        _trg, _ = self.self_attention(trg, trg, trg, trg_mask) #self attention
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))  #dropout, residual connection and layer norm    
        #trg = [batch size, trg len, hid dim]
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask) #encoder attention
        # query, key, value

        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))   #trg = [batch size, trg len, hid dim]

        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))  #trg = [batch size, trg len, hid dim]

        #attention = [batch size, n heads, trg len, src len]
        return trg, attention

In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [27]:
len(word_map)


448

In [28]:
INPUT_DIM = len(word_map)
OUTPUT_DIM = len(word_map)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)


In [29]:
# SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, word_map['<pad>'], word_map['<pad>'], device).to(device)


In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,298,176 trainable parameters


In [31]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(
      (embed): Embedding(448, 256)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, i

In [32]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)  

In [33]:
criterion = nn.CrossEntropyLoss(ignore_index = word_map['<pad>'])


In [34]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0].to(device)
        trg = batch[1].to(device)
        

        #print("Src and trg shape " , src.shape , trg.shape)
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[0]

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 0s
	Train Loss: 3.545 | Train PPL:  34.623
Epoch: 02 | Time: 0m 0s
	Train Loss: 3.440 | Train PPL:  31.194
Epoch: 03 | Time: 0m 0s
	Train Loss: 3.348 | Train PPL:  28.452
Epoch: 04 | Time: 0m 0s
	Train Loss: 3.259 | Train PPL:  26.031
Epoch: 05 | Time: 0m 0s
	Train Loss: 3.182 | Train PPL:  24.103
Epoch: 06 | Time: 0m 0s
	Train Loss: 3.113 | Train PPL:  22.477
Epoch: 07 | Time: 0m 0s
	Train Loss: 3.031 | Train PPL:  20.710
Epoch: 08 | Time: 0m 0s
	Train Loss: 2.955 | Train PPL:  19.195
Epoch: 09 | Time: 0m 0s
	Train Loss: 2.884 | Train PPL:  17.879
Epoch: 10 | Time: 0m 0s
	Train Loss: 2.839 | Train PPL:  17.093
Epoch: 11 | Time: 0m 0s
	Train Loss: 2.768 | Train PPL:  15.933
Epoch: 12 | Time: 0m 0s
	Train Loss: 2.710 | Train PPL:  15.027
Epoch: 13 | Time: 0m 0s
	Train Loss: 2.642 | Train PPL:  14.045
Epoch: 14 | Time: 0m 0s
	Train Loss: 2.584 | Train PPL:  13.247
Epoch: 15 | Time: 0m 0s
	Train Loss: 2.536 | Train PPL:  12.635
Epoch: 16 | Time: 0m 0s
	Train Loss: 2.4

In [66]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = ['<start>'] + tokens + ['<end>']
        
    print(tokens)
    src_indexes = [src_field[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = trg_field['<start>']
    print(trg_indexes)

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field['<end>']:
            break
    
    trg_tokens = [trg_field[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [67]:
#word_map